# 1. Initialisation

## 1.1. File System

In [1]:
from google.colab import drive
drive.mount("/content/drive")

! chmod -R 777 'drive/My Drive/Otago/F. The Thesis/SEmHuS'

% cd 'drive/My Drive/Otago/F. The Thesis/SEmHuS'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Otago/F. The Thesis/SEmHuS


## 1.2. Importing Libraries

In [2]:
# !pip install livelossplot
# !pip install pandas-profiling==2.7.1
# !pip uninstall -y h5py
!pip install 'h5py==2.10.0' --force-reinstall

  Using cached h5py-2.10.0-cp37-cp37m-manylinux1_x86_64.whl (2.9 MB)
  Using cached numpy-1.21.4-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.4
    Uninstalling numpy-1.21.4:
      Successfully uninstalled numpy-1.21.4
  Attempting uninstall: h5py
    Found existing installation: h5py 2.10.0
    Uninstalling h5py-2.10.0:
      Successfully uninstalled h5py-2.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.21.4 which is incompatible.
google-colab 1.0.0 requires six~=1.15.0, but you have six 1.16.

In [3]:
import pandas as pd
import pandas_profiling
from pandas_profiling import ProfileReport

import time
import re
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import nltk
import csv
import ast
import pickle
import itertools
import random
from scipy.spatial import distance

In [4]:
%tensorflow_version 1.x
import tensorflow as tf
from keras import Model
from keras.backend.tensorflow_backend import set_session
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn import metrics
from keras.preprocessing.sequence import pad_sequences

# from sklearn import preprocessing

# from keras.preprocessing.text import one_hot
# from tensorflow.keras.preprocessing.text import Tokenizer
# from keras.utils import to_categorical
# from keras import Input
# from keras.layers import Dense, Concatenate, BatchNormalization
# from keras.layers import Flatten, Dot
# from keras.layers import LSTM, GRU, Dropout
# from keras.layers import Lambda
# from keras.layers import Activation
# from keras.layers.embeddings import Embedding
import keras.backend as K
from keras.losses import mse

TensorFlow 1.x selected.


Using TensorFlow backend.


In [5]:
from keras.models import load_model

In [6]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim.models.keyedvectors import KeyedVectors

In [7]:
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
sys.path.insert(0, './Libraries')
import preprocess

## 1.3. GPU Initialisation

In [9]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device



## 1.4. Global Variables

In [10]:
# model_name = 'RIVEC300_'
model_name = 'RIVEC300_SMALL_'
max_word = 3500
num_ent = 100
alpha = -1.5

In [11]:
Use_Word2Vec = True

## 1.5. Model Reloading

In [12]:
def custom_loss_func(first_pos, second_pos):
    def custom_loss(y_true, y_pred):
        first_idx = K.argmax(K.abs(first_pos), axis = -1)
        second_idx = K.argmax(K.abs(second_pos), axis = -1)
        
        first_max = tf.gather(first_pos, first_idx, axis = 1)
        second_max = tf.gather(second_pos, second_idx, axis = 1)

        first_second = K.mean(mse(first_max, second_max)) * 0.1
        first_first = K.binary_crossentropy(y_true, y_pred)

        return mse(first_second, first_first)
    custom_loss.__name__ = 'custom_loss'
    return custom_loss

In [13]:
# if Use_Word2Vec:
#     word_vectors = KeyedVectors.load_word2vec_format('./Pretrained/GoogleNews-vectors-negative300.bin', binary=True)

In [14]:
# if Use_Word2Vec: w2v_ext = 'With_Word2Vec'
# else: w2v_ext = 'Without_Word2Vec'

In [15]:
modelInputFile = f'./Model/{model_name}SEmHus_Word_Embedding_Model.h5'
model = load_model(modelInputFile)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



## 1.6. Tokenizers Reloading

In [16]:
with open(f'./Tokenizer/{model_name}text.pickle', 'rb') as handle:
    texts_tok = pickle.load(handle)

with open(f'./Tokenizer/{model_name}actor.pickle', 'rb') as handle:
    labActor_tok = pickle.load(handle)

with open(f'./Tokenizer/{model_name}agency.pickle', 'rb') as handle:
    labAgency_tok = pickle.load(handle)

with open(f'./Tokenizer/{model_name}sector.pickle', 'rb') as handle:
    labSector_tok = pickle.load(handle)

with open(f'./Tokenizer/{model_name}place.pickle', 'rb') as handle:
    labPlace_tok = pickle.load(handle)

with open(f'./Tokenizer/{model_name}year.pickle', 'rb') as handle:
    labYear_tok = pickle.load(handle)

with open(f'./Tokenizer/{model_name}month.pickle', 'rb') as handle:
    labMonth_tok = pickle.load(handle)

with open(f'./Tokenizer/{model_name}reason.pickle', 'rb') as handle:
    labReason_tok = pickle.load(handle)

with open(f'./Tokenizer/{model_name}goal.pickle', 'rb') as handle:
    labGoal_tok = pickle.load(handle)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator MultiLabelBinarizer from version 0.22.2.post1 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


# 2. Data Preparation

In [17]:
def clean_alpha(text):
    text = ' '.join([w for w in str(text).split() if w.isalpha()])
    return text

## 2.1. Input and Output

In [18]:
# normInputFile = './Inputs/SEmHus_Enriched_Normalised_Dataset.csv'
normInputFile = f'./Inputs/{model_name}SEmHus_Enriched_Normalised_Dataset_Test.csv'
normDF = pd.read_csv(normInputFile)

In [19]:
normDF = normDF.loc[(normDF['country_name'] != "['World']") & (normDF['disaster_type'] != "['Other']") & (normDF['source_type'] != "['Media']")]

In [20]:
docs = normDF['text'].tolist()
labActor = [eval(v) for v in normDF['source_name'].tolist()]
labAgency = [eval(v) for v in normDF['source_type'].tolist()]
labSector = [eval(v) for v in normDF['theme'].tolist()]
labPlace = [eval(v) for v in normDF['country_name'].tolist()]
labYear = [eval(v) for v in normDF['year_created'].tolist()]
labMonth = [eval(v) for v in normDF['month_created'].tolist()]
labReason = [eval(v) for v in normDF['disaster_type'].tolist()]
labGoal = [eval(v) for v in normDF['development_goal'].tolist()]

In [21]:
labPlace

[['Myanmar'],
 ['Marshall Islands'],
 ['Vanuatu'],
 ['India'],
 ['Burkina Faso'],
 ['Afghanistan'],
 ['Malawi'],
 ['Liberia'],
 ['United States of America'],
 ['Gambia'],
 ['Malawi'],
 ['Liberia'],
 ['Pakistan'],
 ['Philippines'],
 ['Indonesia'],
 ['Somalia'],
 ['India'],
 ['Sri Lanka'],
 ['Indonesia'],
 ["Democratic People's Republic of Korea"],
 ['Philippines'],
 ['Mali'],
 ['Niger'],
 ['Japan'],
 ['Pakistan'],
 ['Mali'],
 ['Bangladesh'],
 ['Philippines'],
 ['Pakistan'],
 ['Liberia'],
 ['Indonesia'],
 ['Indonesia'],
 ['Costa Rica'],
 ['Ethiopia'],
 ['China'],
 ['Djibouti'],
 ['United States of America'],
 ['Pakistan'],
 ['India'],
 ['Indonesia'],
 ['Indonesia'],
 ['Mali'],
 ['Somalia'],
 ['Mexico'],
 ['Niger'],
 ['Haiti'],
 ['Pakistan'],
 ['Philippines'],
 ['Haiti'],
 ['Philippines'],
 ['Indonesia'],
 ['Madagascar'],
 ['Pakistan'],
 ['Philippines'],
 ['Ethiopia'],
 ['Haiti'],
 ['Guyana'],
 ['Nicaragua'],
 ['Bangladesh'],
 ['Fiji'],
 ['Philippines'],
 ['Myanmar'],
 ['India'],
 ['Iraq'

In [22]:
labReason_tok.classes_

array(['Cold Wave', 'Drought', 'Earthquake', 'Epidemic',
       'Extratropical Cyclone', 'Fire', 'Flash Flood', 'Flood',
       'Heat Wave', 'Insect Infestation', 'Land Slide', 'Mud Slide',
       'Other', 'Severe Local Storm', 'Snow Avalanche', 'Storm Surge',
       'Technological Disaster', 'Tropical Cyclone', 'Tsunami', 'Volcano',
       'Wild Fire'], dtype=object)

In [23]:
# lbl_sec = labSector_tok.classes_.astype('U')
# np.savetxt('./Lists/sector_list.csv', lbl_sec, delimiter=',', fmt='%s')

In [24]:
# lbl_sec = labReason_tok.classes_.astype('U')
# np.savetxt('./Lists/reason_list.csv', lbl_sec, delimiter=',', fmt='%s')

In [25]:
docs = [' '.join(str(d).lower().split()[:max_word]) for d in docs]

In [26]:
len(labActor)

5280

In [27]:
padding_len = model.layers[0].get_output_at(0).get_shape().as_list()[1]

In [28]:
padding_len

100

## 2.2. Import Dictionaries

In [29]:
list_encoded_text = f'./Dictionary/{model_name}list_encoded_text.csv'
word_embed_dict = f'./Dictionary/{model_name}word_embed_dict.csv'

# 3. Class Embedding

## 3.1. Semantic Similarity Setup

In [30]:
# from sklearn.metrics.pairwise import cosine_similarity

# # Vectors
# vec_a = [1, 2, 3, 4, 5]
# vec_b = [1, 3, 5, 7, 9]

# # Dot and norm
# dot = sum(a*b for a, b in zip(vec_a, vec_b))
# norm_a = sum(a*a for a in vec_a) ** 0.5
# norm_b = sum(b*b for b in vec_b) ** 0.5

# # Cosine similarity
# cos_sim = dot / (norm_a*norm_b)

# # Results
# print('My version:', cos_sim)
# print('Scikit-Learn:', cosine_similarity([vec_a], [vec_b]))

In [31]:
def cosine(vec1, vec2):
    vec1 = np.array(vec1).tolist()
    vec2 = np.array(vec2).tolist()
    dot = sum(a*b for a, b in zip(vec1, vec2))
    norm_a = sum(a*a for a in vec1) ** 0.5
    norm_b = sum(b*b for b in vec2) ** 0.5
    cos_sim = dot / (norm_a*norm_b)
    return cos_sim

In [32]:
# num_word = 20

In [33]:
agency_weight = model.get_layer('outAgency').get_weights()[0]
agency_transpose = np.asarray(np.transpose(agency_weight))
agency_embed = {w: agency_transpose[idx] for idx, w in enumerate(labAgency_tok.classes_)}

actor_weight = model.get_layer('outActor').get_weights()[0]
actor_transpose = np.asarray(np.transpose(actor_weight))
actor_embed = {w: actor_transpose[idx] for idx, w in enumerate(labActor_tok.classes_)}

goal_weight = model.get_layer('outGoal').get_weights()[0]
goal_transpose = np.asarray(np.transpose(goal_weight))
goal_embed = {w: goal_transpose[idx] for idx, w in enumerate(labGoal_tok.classes_)}

sector_weight = model.get_layer('outSector').get_weights()[0]
sector_transpose = np.asarray(np.transpose(sector_weight))
sector_embed = {w: sector_transpose[idx] for idx, w in enumerate(labSector_tok.classes_)}

place_weight = model.get_layer('outPlace').get_weights()[0]
place_transpose = np.asarray(np.transpose(place_weight))
place_embed = {w: place_transpose[idx] for idx, w in enumerate(labPlace_tok.classes_)}

year_weight = model.get_layer('outYear').get_weights()[0]
year_transpose = np.asarray(np.transpose(year_weight))
year_embed = {w: year_transpose[idx] for idx, w in enumerate(labYear_tok.classes_)}

month_weight = model.get_layer('outMonth').get_weights()[0]
month_transpose = np.asarray(np.transpose(month_weight))
month_embed = {w: month_transpose[idx] for idx, w in enumerate(labMonth_tok.classes_)}

reason_weight = model.get_layer('outReason').get_weights()[0]
reason_transpose = np.asarray(np.transpose(reason_weight))
reason_embed = {w: reason_transpose[idx] for idx, w in enumerate(labReason_tok.classes_)}

In [34]:
# def nearest_label(group, entity):
#     if group == '*':
#         new_df = class_df
#     else:
#         new_df = class_df.loc[class_df['Class'] == group]

#     vec1 = new_df.loc[class_df['Entity'] == entity]
#     vec1 = vec1.iloc[0]['Vector']
#     dist = []
#     # for cls, ent, vec2 in class_data:
#     for cls, ent, vec2 in class_df.values.tolist():
#         d = distance.cosine(vec1, vec2)
#         if d > 0:
#             dist.append((round(distance.cosine(vec1, vec2), 4), ent, cls))
#     dist.sort()
#     dist = dist[:20]
#     return dist

In [35]:
# def nearest_class(first, second, third, num_ent):
#     if first == 'Agency': class_list = agency_embed.items()
#     if first == 'Actor': class_list = actor_embed.items()
#     if first == 'Goal': class_list = goal_embed.items()
#     if first == 'Sector': class_list = sector_embed.items()
#     if first == 'Place': class_list = place_embed.items()
#     if first == 'Year': class_list = year_embed.items()
#     if first == 'Month': class_list = month_embed.items()
#     if first == 'Reason': class_list = reason_embed.items()

#     if second == 'Agency': vec1 = agency_embed[third]
#     if second == 'Actor': vec1 = actor_embed[third]
#     if second == 'Goal': vec1 = goal_embed[third]
#     if second == 'Sector': vec1 = sector_embed[third]
#     if second == 'Place': vec1 = place_embed[third]
#     if second == 'Year': vec1 = year_embed[third]
#     if second == 'Month': vec1 = month_embed[third]
#     if second == 'Reason': vec1 = reason_embed[third]

#     dist = []
#     for word, vec2 in class_list:
#         d = distance.cosine(vec1, vec2)
#         if d > 0:
#             #dist.append((np.linalg.norm(vec1 - vec2), word))
#             dist.append((round(d, 4), word))
#     dist.sort()
#     dist = dist[:num_ent]
#     return dist

## 3.2. Vector Space Export

In [36]:
def normalize(word_vec):
    norm=np.linalg.norm(word_vec)
    if norm == 0:
       return word_vec
    return word_vec/norm

In [37]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)

In [38]:
cols = ['Class', 'Entity', 'Vector', 'X', 'Y']
rows = []

ent = []
nam = []
vec = []

for key, value in agency_embed.items():
    ent.append('Agency')
    nam.append(key)
    normed = normalize(list(value))
    vec.append(normed)

for key, value in actor_embed.items():
    ent.append('Actor')
    nam.append(key)
    normed = normalize(list(value))
    vec.append(normed)

for key, value in goal_embed.items():
    ent.append('Goal')
    nam.append(key)
    normed = normalize(list(value))
    vec.append(normed)

for key, value in sector_embed.items():
    ent.append('Sector')
    nam.append(key)
    normed = normalize(list(value))
    vec.append(normed)

for key, value in place_embed.items():
    ent.append('Place')
    nam.append(key)
    normed = normalize(list(value))
    vec.append(normed)

for key, value in year_embed.items():
    ent.append('Year')
    nam.append(key)
    normed = normalize(list(value))
    vec.append(normed)

for key, value in month_embed.items():
    ent.append('Month')
    nam.append(key)
    normed = normalize(list(value))
    vec.append(normed)

for key, value in reason_embed.items():
    ent.append('Reason')
    nam.append(key)
    normed = normalize(list(value))
    vec.append(normed)

reduced_vector = tsne.fit_transform(vec)
X_dim = reduced_vector[:,0]
Y_dim = reduced_vector[:,1]

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


In [39]:
rows = np.array([ent, nam, vec, X_dim, Y_dim])

rows_t = rows.T 

class_df = pd.DataFrame(data=rows_t, columns = cols)
class_df.to_csv(f'./Outputs/{model_name}Humanitarian_Class_Vector_Space.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [40]:
# class_data = class_df.values.tolist()

## 3.3. Entity Distance Export

In [41]:
file_name = f'./Outputs/{model_name}Humanitarian_Class_Distance_Space.csv'

if os.path.isfile(file_name):
    dist_df = pd.read_csv(file_name)
else:
    cols = ['Source_Class', 'Source_Entity', 'Target_Class', 'Target_Entity', 'Distance']
    rows = []

    for sidx, source in class_df.iterrows():
        for tidx, target in class_df.iterrows():

            if (source['Class'] == target['Class']) and (source['Entity'] == target['Entity']): continue
            
            # dist = distance.euclidean(source['Vector'], target['Vector'])
            dist = distance.cosine(source['Vector'], target['Vector'])
            rows.append([source['Class'], source['Entity'], target['Class'], target['Entity'], dist])

    dist_df = pd.DataFrame(data=rows, columns = cols)
    dist_df.to_csv(file_name)

In [42]:
# file_name = f'./Outputs/{model_name}Humanitarian_Class_Distance_Space.csv'

# cols = ['Source_Class', 'Source_Entity', 'Target_Class', 'Target_Entity', 'Distance']
# rows = []

# for sidx, source in class_df.iterrows():
#     for tidx, target in class_df.iterrows():

#         if (source['Class'] == target['Class']) and (source['Entity'] == target['Entity']): continue
        
#         # dist = distance.euclidean(source['Vector'], target['Vector'])
#         dist = distance.cosine(source['Vector'], target['Vector'])
#         rows.append([source['Class'], source['Entity'], target['Class'], target['Entity'], dist])

# dist_df = pd.DataFrame(data=rows, columns = cols)
# dist_df.to_csv(file_name)

In [43]:
def merge(list1, list2):
    merged_list = [(list1[i], list2[i]) for i in range(0, len(list1))]
    return merged_list

In [44]:
def nearest_class(first, second, third, near):

    class_list = dist_df[(dist_df['Source_Class'] == first) & (dist_df['Target_Class'] == second) & (dist_df['Target_Entity'] == third) ]

    dist = []

    # nam = []
    # dis = []

    for idx, row in class_list.iterrows():
        d = row[5]
        # d = row[4]
        if d > 0:
            #dist.append((np.linalg.norm(vec1 - vec2), word))
            dist.append((round(d, 4), row['Source_Entity']))
            # nam.append(row['Source_Entity'])
            # dis.append(d)
    
    # dis = normalize(dis)
    # dist = merge(dis, nam)
    
    dist.sort()
    dist = dist[:near]

    return dist

In [45]:
def farest_class(first, second, third, near):

    class_list = dist_df[(dist_df['Source_Class'] == first) & (dist_df['Target_Class'] == second) & (dist_df['Target_Entity'] == third) ]


    dist = []

    # nam = []
    # dis = []
    
    for idx, row in class_list.iterrows():
        d = row[5]
        # d = row[4]
        if d > 0:
            #dist.append((np.linalg.norm(vec1 - vec2), word))
            dist.append((round(d, 4), row['Source_Entity']))
            # nam.append(row['Source_Entity'])
            # dis.append(d)
    
    # dis = normalize(dis)
    # dist = merge(dis, nam)

    dist.sort()
    dist = dist[-near:]

    return dist

In [46]:
def find_stat(cls):
    if cls == '*':
        temp_df = dist_df[(dist_df['Distance'] > 0)]
    else:
        temp_df = dist_df[(dist_df['Source_Class'] == cls) & (dist_df['Distance'] > 0)]

    dist_lst = temp_df['Distance'].values

    sd = np.std(dist_lst)
    me = np.mean(dist_lst)
    mx = np.max(dist_lst)
    mn = np.min(dist_lst)
    cs = (mx - mn) / sd

    return sd, me, mx, mn, cs

In [47]:
def find_rank(cls, val, alpha):
    
    stat = find_stat(cls)
    
    stdev = stat[0]
    mean = stat[1]
    # measure = mean - (alpha * stdev)
    measure = (val - mean) / stdev

    # if alpha < measure: rank = 'Accepted'
    # if alpha >= measure: rank = 'Rejected'

    if alpha >= measure: rank = 'Accepted'
    if alpha < measure: rank = 'Rejected'

    return rank

In [48]:
# class_df = dist_df[(dist_df['Source_Class'] == 'Place') & (dist_df['Distance'] > 0)]
# # class_df = dist_df[(dist_df['Distance'] > 0)]

# dist_lst = class_df['Distance'].values

cls = '*'
stat = find_stat(cls)

print('Standard Deviation:', round(stat[0], 6))
print('Mean Distance:', round(stat[1], 6))
print('Maximum Distance:', round(stat[2], 6))
print('Minimum Distance :', round(stat[3], 6))
print('Number of CS :', round(stat[4], 6))

Standard Deviation: 0.10367
Mean Distance: 0.953508
Maximum Distance: 1.444925
Minimum Distance : 0.033215
Number of CS : 13.617288


# 4. Result Validation

In [49]:
# var1, var2 = 'Year', 'Year'

# if var2 == 'Agency': rand_class = labAgency_tok.classes_[random.randint(0, len(labAgency_tok.classes_)-1)]
# if var2 == 'Actor': rand_class = labActor_tok.classes_[random.randint(0, len(labActor_tok.classes_)-1)]
# if var2 == 'Goal': rand_class = labGoal_tok.classes_[random.randint(0, len(labGoal_tok.classes_)-1)]
# if var2 == 'Sector': rand_class = labSector_tok.classes_[random.randint(0, len(labSector_tok.classes_)-1)]
# if var2 == 'Place': rand_class = labPlace_tok.classes_[random.randint(0, len(labPlace_tok.classes_)-1)]
# if var2 == 'Year': rand_class = labYear_tok.classes_[random.randint(0, len(labYear_tok.classes_)-1)]
# if var2 == 'Month': rand_class = labMonth_tok.classes_[random.randint(0, len(labMonth_tok.classes_)-1)]
# if var2 == 'Reason': rand_class = labReason_tok.classes_[random.randint(0, len(labReason_tok.classes_)-1)]

# rand_class = '1996'

# ents = nearest_class(var1, var2, rand_class, num_ent)

# accepted_list = ''
# rejected_list = ''

# acc_ent = []
# rej_ent = []

# nearest_list = ''

# en, dt, sd, me = [], [], [], []

# # # cls = '*'
# cls = var1

# # alpha = -1.0

# stat = find_stat(cls)

# for ent in ents:
#     rank = find_rank(cls, ent[0], alpha)
#     # if rank == 'Accepted':
#     en.append(ent[1])
#     dt.append(ent[0])
#     sd.append(stat[0])
#     me.append(stat[1])
#     d = round(ent[0], 4)
#     z = round((ent[0] - stat[1]) / stat[0], 4)
#     if rank == 'Accepted':
#         accepted_list += f'{ent[1]} [Distance={d}, Z-Score={z}] -- [{rank}];\n'
#         acc_ent.append(ent[1])
#     else:
#         rejected_list += f'{ent[1]} [Distance={d}, Z-Score={z}] -- [{rank}];\n'
#         rej_ent.append(ent[1])
    
#     nearest_list += f'{ent[1]} ({z}); '
#     # nearest_list += f'{ent[1]} ({d}); '

# # nearest_title = f'Nearest {var1}(s) to {var2} [{rand_class}] are {len(en)}:\n'
# nearest_title = f'Nearest {var1}(s) to {var2} [{rand_class}] are {len(en)}:\n'

# print(nearest_title)
# print(nearest_list)

In [50]:
# break

## 4.1. Mapping Table

In [51]:
from sqlite3 import connect
conn = connect(':memory:')

In [52]:
sqlFile = f'./Inputs//{model_name}SEmHus_Enriched_Normalised_Dataset_Test.csv'
# sqlFile = f'./Inputs//SEmHus_Enriched_Normalised_Dataset.csv'
sqlDF = pd.read_csv(sqlFile)
# sqlDF = normDF

In [53]:
# labPlace

In [54]:
places = [p for [p] in labPlace]
# places

In [55]:
# sqlDF['disaster_type'].unique()

In [56]:
labReason_tok.classes_

array(['Cold Wave', 'Drought', 'Earthquake', 'Epidemic',
       'Extratropical Cyclone', 'Fire', 'Flash Flood', 'Flood',
       'Heat Wave', 'Insect Infestation', 'Land Slide', 'Mud Slide',
       'Other', 'Severe Local Storm', 'Snow Avalanche', 'Storm Surge',
       'Technological Disaster', 'Tropical Cyclone', 'Tsunami', 'Volcano',
       'Wild Fire'], dtype=object)

In [57]:
# sqlDF.loc[sqlDF.Disaster_Type == 'Volcanic activity', 'Disaster_Type'] = 'Volcano'
# sqlDF.loc[sqlDF.Disaster_Type == 'Landslide', 'Disaster_Type'] = 'Land Slide'
# sqlDF.loc[sqlDF.Disaster_Type == 'Wildfire', 'Disaster_Type'] = 'Wild Fire'
# sqlDF.loc[sqlDF.Disaster_Type == 'Storm', 'Disaster_Type'] = 'Storm Surge'
# sqlDF.loc[sqlDF.Disaster_Type == 'Insect infestation', 'Disaster_Type'] = 'Insect Infestation'
# sqlDF.loc[sqlDF.Disaster_Type == 'Mass movement (dry)', 'Disaster_Type'] = 'Land Slide'

# sqlDF.loc[sqlDF.Disaster_Subtype == 'Cold wave', 'Disaster_Type'] = 'Cold Wave'
# sqlDF.loc[sqlDF.Disaster_Subtype == 'Heat wave', 'Disaster_Type'] = 'Heat Wave'
# sqlDF.loc[sqlDF.Disaster_Subtype == 'Avalanche', 'Disaster_Type'] = 'Snow Avalanche'
# sqlDF.loc[sqlDF.Disaster_Subtype == 'Mudslide', 'Disaster_Type'] = 'Mud Slide'
# sqlDF.loc[sqlDF.Disaster_Subtype == 'Tropical cyclone', 'Disaster_Type'] = 'Tropical Cyclone'
# sqlDF.loc[sqlDF.Disaster_Subtype == 'Rockfall', 'Disaster_Type'] = 'Land Slide'
# sqlDF.loc[sqlDF.Disaster_Subtype == 'Severe winter conditions', 'Disaster_Type'] = 'Cold Wave'

# # sqlDF.loc[sqlDF.Associated_Dis == 'Tsunami/Tidal wave', 'Disaster_Type'] = 'Tsunami'

In [58]:
# sqlDF

In [59]:
sqlDF.to_sql('Hum_Hist', conn)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:2615: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [60]:
unwanted_list = ['Other']

In [61]:
var1, var2 = 'Reason', 'Place'

if var2 == 'Agency': rand_class = labAgency_tok.classes_[random.randint(0, len(labAgency_tok.classes_)-1)]
if var2 == 'Actor': rand_class = labActor_tok.classes_[random.randint(0, len(labActor_tok.classes_)-1)]
if var2 == 'Goal': rand_class = labGoal_tok.classes_[random.randint(0, len(labGoal_tok.classes_)-1)]
if var2 == 'Sector': rand_class = labSector_tok.classes_[random.randint(0, len(labSector_tok.classes_)-1)]
if var2 == 'Place': rand_class = labPlace_tok.classes_[random.randint(0, len(labPlace_tok.classes_)-1)]
if var2 == 'Year': rand_class = labYear_tok.classes_[random.randint(0, len(labYear_tok.classes_)-1)]
if var2 == 'Month': rand_class = labMonth_tok.classes_[random.randint(0, len(labMonth_tok.classes_)-1)]
if var2 == 'Reason': rand_class = labReason_tok.classes_[random.randint(0, len(labReason_tok.classes_)-1)]

# rand_class = labPlace_tok.classes_[random.randint(0, len(labPlace_tok.classes_)-1)]
rand_class = 'Sudan'
# rand_class = 'United Kingdom of Great Britain and Northern Ireland'
# rand_class = 'Cook Islands'

ents = nearest_class(var1, var2, rand_class, num_ent)

accepted_list = ''
rejected_list = ''

acc_ent = []
rej_ent = []

en, dt, sd, me = [], [], [], []

# cls = '*'
cls = var1

alpha = -2.0

stat = find_stat(cls)

for ent in ents:
    if ent[1] in unwanted_list:
        # print(ent[1])
        continue
    rank = find_rank(cls, ent[0], alpha)
    # if rank == 'Accepted':
    en.append(ent[1])
    dt.append(ent[0])
    sd.append(stat[0])
    me.append(stat[1])
    d = round(ent[0], 4)
    z = round((ent[0] - stat[1]) / stat[0], 4)
    if rank == 'Accepted':
        accepted_list += f'{ent[1]} [Distance={d}, Z-Score={z}] -- [{rank}];\n'
        acc_ent.append(ent[1])
    else:
        rejected_list += f'{ent[1]} [Distance={d}, Z-Score={z}] -- [{rank}];\n'
        rej_ent.append(ent[1])
    # nearest_list += f'{ent[1]} ({z}); '
    # nearest_list += f'{ent[1]} ({d}); \n'

# nearest_title = f'Nearest {var1}(s) to {var2} [{rand_class}] are {len(en)}:\n'
nearest_title = f'Nearest {var1}(s) to {var2} [{rand_class}] are {len(en)}:\n'

print(nearest_title)
print('Accepted Results:')
print(accepted_list)
print('Rejected Results:')
print(rejected_list)

Nearest Reason(s) to Place [Sudan] are 20:

Accepted Results:
Flash Flood [Distance=0.8077, Z-Score=-2.1878] -- [Accepted];

Rejected Results:
Storm Surge [Distance=0.8873, Z-Score=-1.2329] -- [Rejected];
Flood [Distance=0.8892, Z-Score=-1.2101] -- [Rejected];
Epidemic [Distance=0.8951, Z-Score=-1.1393] -- [Rejected];
Insect Infestation [Distance=0.9209, Z-Score=-0.8298] -- [Rejected];
Snow Avalanche [Distance=0.9398, Z-Score=-0.603] -- [Rejected];
Cold Wave [Distance=0.9544, Z-Score=-0.4279] -- [Rejected];
Wild Fire [Distance=0.9654, Z-Score=-0.2959] -- [Rejected];
Fire [Distance=1.0085, Z-Score=0.2211] -- [Rejected];
Drought [Distance=1.0118, Z-Score=0.2607] -- [Rejected];
Severe Local Storm [Distance=1.0179, Z-Score=0.3339] -- [Rejected];
Extratropical Cyclone [Distance=1.0263, Z-Score=0.4347] -- [Rejected];
Technological Disaster [Distance=1.0323, Z-Score=0.5067] -- [Rejected];
Earthquake [Distance=1.0329, Z-Score=0.5139] -- [Rejected];
Tsunami [Distance=1.0495, Z-Score=0.713] -- [

In [62]:
# var1, var2 = 'Reason', 'Place'

# if var2 == 'Agency': rand_class = labAgency_tok.classes_[random.randint(0, len(labAgency_tok.classes_)-1)]
# if var2 == 'Actor': rand_class = labActor_tok.classes_[random.randint(0, len(labActor_tok.classes_)-1)]
# if var2 == 'Goal': rand_class = labGoal_tok.classes_[random.randint(0, len(labGoal_tok.classes_)-1)]
# if var2 == 'Sector': rand_class = labSector_tok.classes_[random.randint(0, len(labSector_tok.classes_)-1)]
# if var2 == 'Place': rand_class = labPlace_tok.classes_[random.randint(0, len(labPlace_tok.classes_)-1)]
# if var2 == 'Year': rand_class = labYear_tok.classes_[random.randint(0, len(labYear_tok.classes_)-1)]
# if var2 == 'Month': rand_class = labMonth_tok.classes_[random.randint(0, len(labMonth_tok.classes_)-1)]
# if var2 == 'Reason': rand_class = labReason_tok.classes_[random.randint(0, len(labReason_tok.classes_)-1)]

# rand_class = 'Sudan'

# acc_ent = nearest_class(var1, var2, rand_class, num_ent)
# rej_ent = farest_class(var1, var2, rand_class, num_ent)

# acc_ent = [e for (d, e) in acc_ent]
# rej_ent = [e for (d, e) in rej_ent]

# acc_ent, rej_ent

## 4.2. Using Benchmarks

In [63]:
if var1 == 'Place': source_col = 'country_name'
if var1 == 'Year': source_col = 'year_created'
if var1 == 'Reason': source_col = 'disaster_type'

if var2 == 'Place': target_col = 'country_name'
if var2 == 'Year': target_col = 'year_created'
if var2 == 'Reason': target_col = 'disaster_type'

In [64]:
def sql_return(source_col, target_col, rand_class):
    tempDF = pd.read_sql(f'''SELECT {source_col}, COUNT(disaster_type) AS Freq, {target_col}
                            FROM Hum_Hist
                            WHERE {target_col} = "['{rand_class}']"
                            GROUP BY {source_col}
                            ORDER BY Freq Desc''', conn)
    OneList = []

    for idx, row in tempDF.iterrows():
        OneList.extend(eval(row[source_col]))

    OneList = list(set(OneList))

    return OneList

In [65]:
freq_title = f'Nearest {var1}(s) to {var2} [{rand_class}] are:\n'

OneDF = sql_return(source_col, target_col, rand_class)

pd.options.display.max_colwidth = 0

print(freq_title)

OneDF

Nearest Reason(s) to Place [Sudan] are:



['Earthquake',
 'Tsunami',
 'Insect Infestation',
 'Flood',
 'Epidemic',
 'Flash Flood']

In [66]:
inf = []
pre = []

for e in acc_ent:
    x = e.split()
    new = 0
    for y in x:
        if y in OneDF:
            new = 1
        # else:
        #     new = 0

    if new == 1:
        inf.append(e)
    else:
        pre.append(e)

print(freq_title)
print('All Matched: ', inf)
print('Not Matched: ', pre)
print('')

Nearest Reason(s) to Place [Sudan] are:

All Matched:  ['Flash Flood']
Not Matched:  []



## 4.3. Clustering the Results

In [67]:
# from sklearn.cluster import KMeans

In [68]:
# ent_vec = []

# for e in en:
#     rec = class_df.loc[class_df['Entity'] == e]
#     ent_vec.append(rec['Vector'].tolist()[0])

In [69]:
# kmeans = KMeans(n_clusters=2, random_state=0).fit(ent_vec)
# clust = kmeans.labels_
# # kmeans.cluster_centers_ 
# # USE THIS FUNCTION

In [70]:
# clust

In [71]:
# rel = []
# irr = []

# for idx, e in enumerate(acc_ent):
#     if clust[idx] == clust[0]:
#         rel.append(e)
#     else:
#         irr.append(e)

# print(freq_title)
# print('Most Relevant:  ', rel)
# print('Least Relevant: ', irr)
# print('')

## 4.4. Export Results

In [72]:
# listEarthquake = ['Earthquake', 'Tsunami']
# listVolcano = ['Volcano']
# listStorm = ['Severe Local Storm', 'Storm Surge', 'Tropical Cyclone', 'Extratropical Cyclone']
# listTemperature = ['Heat Wave', 'Cold Wave']
# listFlood = ['Flash Flood', 'Flood']
# listLandslide = ['Mud Slide', 'Land Slide', 'Snow Avalanche']
# listDrought = ['Drought']
# listFire = ['Fire', 'Wild Fire']
# listEpidemic = ['Epidemic']
# listInsect = ['Insect Infestation']

In [73]:
New_Reason = list(labReason_tok.classes_)
New_Reason

['Cold Wave',
 'Drought',
 'Earthquake',
 'Epidemic',
 'Extratropical Cyclone',
 'Fire',
 'Flash Flood',
 'Flood',
 'Heat Wave',
 'Insect Infestation',
 'Land Slide',
 'Mud Slide',
 'Other',
 'Severe Local Storm',
 'Snow Avalanche',
 'Storm Surge',
 'Technological Disaster',
 'Tropical Cyclone',
 'Tsunami',
 'Volcano',
 'Wild Fire']

In [74]:
# def dis_cls(lst):
    
#     disType = []

#     for l in lst:
#         if l in listEarthquake: disType.append('Earthquake')
#         if l in listVolcano: disType.append('Volcano')
#         if l in listStorm: disType.append('Storm')
#         if l in listTemperature: disType.append('Extreme Temperature')
#         if l in listFlood: disType.append('Flood')
#         if l in listLandslide: disType.append('Landslide')
#         if l in listDrought: disType.append('Drought')
#         if l in listFire: disType.append('Fire')
#         if l in listEpidemic: disType.append('Epidemic')
#         if l in listInsect: disType.append('Insect')

#     disType = list(set(disType))

#     return disType

In [75]:
# listGeophysical = ['Earthquake', 'Tsunami', 'Volcano']
# listMeteorological = ['Heat Wave', 'Cold Wave', 'Severe Local Storm', 'Storm Surge', 'Tropical Cyclone', 'Extratropical Cyclone']
# listHydrological = ['Flash Flood', 'Flood', 'Mud Slide', 'Land Slide', 'Snow Avalanche']
# listClimatological = ['Drought', 'Fire', 'Wild Fire']
# listBiological = ['Epidemic', 'Insect Infestation']

In [76]:
# def dis_cls(lst):
    
#     disType = []

#     for l in lst:
#         if l in listGeophysical: disType.append('Geophysical')
#         if l in listMeteorological: disType.append('Meteorological')
#         if l in listHydrological: disType.append('Hydrological')
#         if l in listClimatological: disType.append('Climatological')
#         if l in listBiological: disType.append('Biological')

#     disType = list(set(disType))

#     return disType

In [77]:
def SM_Reasons(plc):
    
    var1, var2 = 'Reason', 'Place'

    rand_class = plc

    ents = nearest_class(var1, var2, rand_class, num_ent)

    accepted_list = ''
    rejected_list = ''

    acc_ent = []
    rej_ent = []

    en, dt, sd, me = [], [], [], []

    # cls = '*'
    cls = var1

    # alpha = -1.0

    stat = find_stat(cls)

    for ent in ents:
        if ent[1] in unwanted_list:
            continue
        rank = find_rank(cls, ent[0], alpha)
        # if rank == 'Accepted':
        en.append(ent[1])
        dt.append(ent[0])
        sd.append(stat[0])
        me.append(stat[1])
        d = round(ent[0], 4)
        z = round((ent[0] - stat[1]) / stat[0], 4)
        if rank == 'Accepted':
            accepted_list += f'{ent[1]} [Distance={d}, Z-Score={z}] -- [{rank}];\n'
            acc_ent.append(ent[1])
        else:
            rejected_list += f'{ent[1]} [Distance={d}, Z-Score={z}] -- [{rank}];\n'
            rej_ent.append(ent[1])
        # nearest_list += f'{ent[1]} ({z}); '
        # nearest_list += f'{ent[1]} ({d}); \n'

    # nearest_title = f'Nearest {var1}(s) to {var2} [{rand_class}] are {len(en)}:\n'
    nearest_title = f'Nearest {var1}(s) to {var2} [{rand_class}] are {len(en)}:\n'
    return acc_ent, rej_ent

In [78]:
def EM_Reasons(plc, results):

    accepted, rejected = results[0], results[1]

    rej_temp = []

    for r in rejected:
        if r not in accepted:
            rej_temp.append(r)

    rejected = rej_temp

    tp = []
    fp = []

    tn = []
    fn = []

    bothDF = sql_return(source_col, target_col, plc)

    # print(OneDF)
    # print(twoDF)
    # print(bothDF, '\n')

    if len(bothDF) >= 1:

        for e in accepted:
            if e in bothDF:
                tp.append(e)
            else:
                fp.append(e)

        for e in rejected:
            if e in bothDF:
                fn.append(e)
            else:
                tn.append(e)

    return tp, fp, tn, fn, bothDF

## Signle Country

In [85]:
plc = labPlace_tok.classes_[random.randint(0, len(labPlace_tok.classes_)-1)]
# plc = 'Sudan'
# plc = 'United Kingdom of Great Britain and Northern Ireland'
# plc = rand_class
# plc = 'Romania'

if plc not in places:
    print(f'Sorry, country {plc} not found!\n')
else:
    print(f'Evalaution of {plc}:')

alpha = -2.5

SM_RE = SM_Reasons(plc)
EM_RE = EM_Reasons(plc, SM_RE)

act_sem = SM_RE[0]
act_emd = EM_RE[4]

y_act = []
y_pre = []

for r in New_Reason:
    if r in act_emd and r in act_sem:
        y_act.append(1)
        y_pre.append(1)
    elif r in act_emd and r not in act_sem:
        y_act.append(1)
        y_pre.append(0)
    elif r not in act_emd and r in act_sem:
        y_act.append(0)
        y_pre.append(1)
    else:
        y_act.append(0)
        y_pre.append(0)

# for r in New_Reason:
#     if r in act_sem and r in act_emd:
#         y_act.append(r)
#         y_pre.append(r)
#     elif r in act_sem and r not in act_emd:
#         y_act.append(r)
#         y_pre.append('None')
#     elif r not in act_sem and r in act_emd:
#         y_act.append('None')
#         y_pre.append(r)
#     else:
#         y_act.append('None')
#         y_pre.append('None')

# cls_res = score(y_act, y_pre, average='weighted', zero_division=0)

# cls_pre, cls_rec, cls_f1, cls_sup = cls_res

cls_pre = metrics.precision_score(y_act, y_pre)
cls_rec = metrics.recall_score(y_act, y_pre)
cls_f1 = metrics.f1_score(y_act, y_pre)
cls_hmm = metrics.hamming_loss(y_act, y_pre)

temp_TP = EM_RE[0]
temp_FP = EM_RE[1]
temp_TN = EM_RE[2]
temp_FN = EM_RE[3]

count_TP = len(temp_TP)
count_FP = len(temp_FP)
count_TN = len(temp_TN)
count_FN = len(temp_FN)

# temp_acc = 0
# temp_pre = 0
# temp_rec = 0
# temp_f1 = 0

if (count_TP + count_FP + count_FN + count_TN) == 0:
    temp_acc = 0
else:
    temp_acc = (count_TP + count_TN) / (count_TP + count_FP + count_FN + count_TN)

if count_TP + count_FP == 0:
    temp_pre = 0
else:
    temp_pre = count_TP / (count_TP + count_FP)

if count_TP + count_FN == 0:
    temp_rec = 0
else:
    temp_rec = count_TP / (count_TP + count_FN)

if (temp_rec + temp_pre) == 0:
    temp_f1 = 0
else:
    temp_f1 = 2 * (temp_rec * temp_pre) / (temp_rec + temp_pre)

print('\r')
print(f'All Classes ({len(New_Reason)})\t', New_Reason)

print('\r')
print(f'Result (EM-DAT) ({len(act_emd)})\t', act_emd)
print(f'Result (SEmHuS) ({len(act_sem)})\t', act_sem)

print('\r')
print(f'Y-Act (EM-DAT) ({len(y_act)})\t', y_act)
print(f'Y-Pre (SEmHuS) ({len(y_pre)})\t', y_pre)

print('\r')
print(plc, f'Positive {SM_RE[0]}')
print(plc, f'Negative {SM_RE[1]}')

print('\r')
print(f'True Positive  ({len(temp_TP)})\t', temp_TP)
print(f'False Positive ({len(temp_FP)})\t', temp_FP)

print('\r')
print(f'True Negative  ({len(temp_TN)})\t', temp_TN)
print(f'False Negative ({len(temp_FN)})\t', temp_FN)

# print('\r')
# print(f'Label Accuracy  \t', round(temp_acc * 100, 2), '%')

print('\r')
print(f'Label Precision  \t', round(temp_pre * 100, 2), '%')
print(f'Label Recall  \t\t', round(temp_rec * 100, 2), '%')
print(f'Label F1_score  \t', round(temp_f1 * 100, 2), '%')

print('\r')
print(f'Multi Precision  \t', round(cls_pre * 100, 2), '%')
print(f'Multi Recall  \t\t', round(cls_rec * 100, 2), '%')
print(f'Multi F1_score  \t', round(cls_f1 * 100, 2), '%')

print('\r')
print(f'Hamming Loss  \t\t', round(cls_hmm * 100, 2), '%')

# print(cls_res)

Evalaution of Cuba:

All Classes (21)	 ['Cold Wave', 'Drought', 'Earthquake', 'Epidemic', 'Extratropical Cyclone', 'Fire', 'Flash Flood', 'Flood', 'Heat Wave', 'Insect Infestation', 'Land Slide', 'Mud Slide', 'Other', 'Severe Local Storm', 'Snow Avalanche', 'Storm Surge', 'Technological Disaster', 'Tropical Cyclone', 'Tsunami', 'Volcano', 'Wild Fire']

Result (EM-DAT) (2)	 ['Tropical Cyclone', 'Flood']
Result (SEmHuS) (1)	 ['Tropical Cyclone']

Y-Act (EM-DAT) (21)	 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
Y-Pre (SEmHuS) (21)	 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]

Cuba Positive ['Tropical Cyclone']
Cuba Negative ['Storm Surge', 'Drought', 'Volcano', 'Severe Local Storm', 'Flood', 'Heat Wave', 'Wild Fire', 'Extratropical Cyclone', 'Snow Avalanche', 'Epidemic', 'Insect Infestation', 'Fire', 'Earthquake', 'Mud Slide', 'Cold Wave', 'Technological Disaster', 'Tsunami', 'Land Slide', 'Flash Flood']

True Positive  (1)	 ['Tropical Cyclone']
Fal

In [86]:
# alpha_range = range(-12, 13)
alpha_range = range(-8, 9)
# alpha_range = range(-1, 2)

In [87]:
tot_alpha = []

tot_acc = []
tot_pre = []
tot_rec = []
tot_spc = []
tot_f1 = []

tot_tp = []
tot_fp = []
tot_tn = []
tot_fn = []

all_act = []
all_pre = []

for i in alpha_range:
    
    if i==0: alpha = 0
    else: alpha = i * 0.25

    country = []

    true_positive = []
    false_positive = []
    true_negative = []
    false_negative = []

    tp = []
    fp = []
    tn = []
    fn = []

    alpha_act = []
    alpha_pre = []

    for plc in labPlace_tok.classes_:

        if plc not in places: continue

        SM_RE = SM_Reasons(plc)
        EM_RE = EM_Reasons(plc, SM_RE)

        act_sem = SM_RE[0]
        act_emd = EM_RE[4]

        y_act = []
        y_pre = []

        for r in New_Reason:
            if r in act_emd and r in act_sem:
                y_act.append(1)
                y_pre.append(1)
            elif r in act_emd and r not in act_sem:
                y_act.append(1)
                y_pre.append(0)
            elif r not in act_emd and r in act_sem:
                y_act.append(0)
                y_pre.append(1)
            else:
                y_act.append(0)
                y_pre.append(0)

        # for r in New_Reason:
        #     if r in act_sem and r in act_emd:
        #         y_act.append(r)
        #         y_pre.append(r)
        #     elif r in act_sem and r not in act_emd:
        #         y_act.append(r)
        #         y_pre.append('None')
        #     elif r not in act_sem and r in act_emd:
        #         y_act.append('None')
        #         y_pre.append(r)
        #     else:
        #         y_act.append('None')
        #         y_pre.append('None')

        alpha_act.append(y_act)
        alpha_pre.append(y_pre)

        num_tp = len(EM_RE[0])
        num_fp = len(EM_RE[1])
        num_tn = len(EM_RE[2])
        num_fn = len(EM_RE[3])

        num_sum = num_tp + num_fp + num_tn + num_fn
        if num_sum == 0: continue

        country.append(plc)

        true_positive.append(EM_RE[0])
        false_positive.append(EM_RE[1])
        true_negative.append(EM_RE[2])
        false_negative.append(EM_RE[3])

        tp.append(num_tp)
        fp.append(num_fp)
        tn.append(num_tn)
        fn.append(num_fn)
    
    resultDict = {'Country': country, 'True Positive': true_positive, 'False Positive': false_positive, 'True Negative': true_negative, 'False Negative': false_negative, 'TP': tp, 'FP': fp, 'TN': tn, 'FN': fn}
    resultDF = pd.DataFrame(data=resultDict)
    val_result = f'./Outputs/{model_name}Validated_Results_EM_DAT_Alpha_{alpha}.csv'
    resultDF.to_csv(val_result)

    TP = resultDF['TP'].sum()
    FP = resultDF['FP'].sum()
    TN = resultDF['TN'].sum()
    FN = resultDF['FN'].sum()

    Accuracy = (TP+TN) / (TP+FP+FN+TN)
    Precision = TP / (TP+FP)
    Recall = TP / (TP+FN)
    F1_Score = 2 * (Recall * Precision) / (Recall + Precision)

    Accuracy = round(Accuracy * 100, 2)
    Precision = round(Precision * 100, 2)
    Recall = round(Recall * 100, 2)
    F1_Score = round(F1_Score * 100, 2)

    print(f'Alpha Value \t: {alpha}')
    print(f'Accuracy \t: {Accuracy} %')
    print(f'Precision \t: {Precision} %')
    print(f'Recall \t\t: {Recall} %')
    print(f'F1 Score \t: {F1_Score} %\n')

    tot_tp.append(TP)
    tot_fp.append(FP)
    tot_tn.append(TN)
    tot_fn.append(FN)

    tot_alpha.append(alpha)
    tot_acc.append(Accuracy)
    tot_pre.append(Precision)
    tot_rec.append(Recall)
    tot_f1.append(F1_Score)

    all_act.append(alpha_act)
    all_pre.append(alpha_pre)

resDict = {'Alpha': tot_alpha, 'TP': tot_tp, 'FP': tot_fp, 'TN': tot_tn, 'FN': tot_fn, 'Accuracy': tot_acc, 'Precision': tot_pre, 'Recall': tot_rec, 'F1': tot_f1}
resDF = pd.DataFrame(data=resDict)
res_file = f'./Outputs/{model_name}Accuracy_Precision_Recall_F1_Results.csv'
resDF.to_csv(res_file)

Alpha Value 	: -2.0
Accuracy 	: 83.61 %
Precision 	: 76.47 %
Recall 		: 24.07 %
F1 Score 	: 36.62 %

Alpha Value 	: -1.75
Accuracy 	: 83.71 %
Precision 	: 71.98 %
Recall 		: 28.11 %
F1 Score 	: 40.44 %

Alpha Value 	: -1.5
Accuracy 	: 83.51 %
Precision 	: 65.79 %
Recall 		: 33.67 %
F1 Score 	: 44.54 %

Alpha Value 	: -1.25
Accuracy 	: 82.72 %
Precision 	: 59.05 %
Recall 		: 39.56 %
F1 Score 	: 47.38 %

Alpha Value 	: -1.0
Accuracy 	: 80.79 %
Precision 	: 51.35 %
Recall 		: 44.95 %
F1 Score 	: 47.94 %

Alpha Value 	: -0.75
Accuracy 	: 77.22 %
Precision 	: 43.07 %
Recall 		: 49.16 %
F1 Score 	: 45.91 %

Alpha Value 	: -0.5
Accuracy 	: 73.31 %
Precision 	: 37.87 %
Recall 		: 55.72 %
F1 Score 	: 45.1 %

Alpha Value 	: -0.25
Accuracy 	: 67.78 %
Precision 	: 33.19 %
Recall 		: 62.96 %
F1 Score 	: 43.46 %

Alpha Value 	: 0
Accuracy 	: 61.56 %
Precision 	: 29.94 %
Recall 		: 71.21 %
F1 Score 	: 42.15 %

Alpha Value 	: 0.25
Accuracy 	: 53.77 %
Precision 	: 26.66 %
Recall 		: 77.1 %
F1 Score 	: 

In [88]:
li = []

for i in alpha_range:
    if i==0: alpha = 0
    else: alpha = i * -0.25
    alpha_file = f'./Outputs/{model_name}Validated_Results_EM_DAT_Alpha_{alpha}.csv'
    df = pd.read_csv(alpha_file, index_col=None, header=0)
    df['Alpha'] = np.repeat(alpha, len(df))
    df['Accuracy'] = ((df['TP']+df['TN']) / (df['TP']+df['FP']+df['FN']+df['TN']))
    df['Precision'] = (df['TP'] / (df['TP']+df['FP']))
    df['Recall'] = (df['TP'] / (df['TP']+df['FN']))
    df['F1_Score'] = 2 * (df['Recall'] * df['Precision']) / (df['Recall'] + df['Precision'])
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
comb_file = f'./Outputs/{model_name}Validated_Results_EM_DAT_Alpha_All.csv'
frame.to_csv(comb_file)

In [89]:
idx = 0

sum_alpha, sum_pre, sum_rec, sum_f1, sum_y_act, sum_y_pre = [], [], [], [], [], []

for i in alpha_range:
    # cls_res = score(all_act[idx], all_pre[idx], average='weighted', zero_division=0)
    # cls_pre, cls_rec, cls_f1, cls_sup = cls_res

    y_act, y_pre = all_act[idx], all_pre[idx]

    cls_pre = metrics.precision_score(y_act, y_pre, average='weighted')
    cls_rec = metrics.recall_score(y_act, y_pre, average='weighted')
    cls_f1 = metrics.f1_score(y_act, y_pre, average='weighted')

    alpha_val = i * 0.25

    sum_alpha.append(alpha_val)
    sum_pre.append(cls_pre)
    sum_rec.append(cls_rec)
    sum_f1.append(cls_f1)
    sum_y_pre.append(y_act)
    sum_y_act.append(y_pre)

    print('\r')
    print(f'@ Alpha Value : {alpha_val}')
    print(f'Class Precision  \t', round(cls_pre * 100, 2), '%')
    print(f'Class Recall  \t\t', round(cls_rec * 100, 2), '%')
    print(f'Class F1_score  \t', round(cls_f1 * 100, 2), '%')

    idx += 1

clsDict = {'Alpha': sum_alpha, 'Precision': sum_pre, 'Recall': sum_rec, 'F1 Score': sum_f1, 'Actual':sum_y_act, 'Predicted':sum_y_pre}
clsDF = pd.DataFrame(data=clsDict)
cls_file = f'./Outputs/{model_name}Classified_Metrics_Output_Results.csv'
clsDF.to_csv(cls_file)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Pr


@ Alpha Value : -2.0
Class Precision  	 83.94 %
Class Recall  		 22.73 %
Class F1_score  	 31.45 %

@ Alpha Value : -1.75
Class Precision  	 82.91 %
Class Recall  		 26.55 %
Class F1_score  	 35.29 %

@ Alpha Value : -1.5
Class Precision  	 79.68 %
Class Recall  		 31.8 %
Class F1_score  	 40.16 %

@ Alpha Value : -1.25
Class Precision  	 77.05 %
Class Recall  		 37.36 %
Class F1_score  	 44.79 %

@ Alpha Value : -1.0
Class Precision  	 70.74 %
Class Recall  		 42.45 %
Class F1_score  	 47.87 %

@ Alpha Value : -0.75
Class Precision  	 65.32 %
Class Recall  		 46.42 %
Class F1_score  	 48.27 %

@ Alpha Value : -0.5
Class Precision  	 61.16 %
Class Recall  		 52.62 %
Class F1_score  	 50.94 %

@ Alpha Value : -0.25
Class Precision  	 58.56 %
Class Recall  		 59.46 %
Class F1_score  	 53.44 %

@ Alpha Value : 0.0
Class Precision  	 55.33 %
Class Recall  		 67.25 %
Class F1_score  	 55.6 %

@ Alpha Value : 0.25
Class Precision  	 51.47 %
Class Recall  		 72.81 %
Class F1_score  	 55.42 %

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Pr

In [90]:
# val_result = f'./Outputs/{model_name}Validated_Results_EM_DAT_Alpha_{alpha}.csv'

In [91]:
# country = []

# true_positive = []
# false_positive = []

# true_negative = []
# false_negative = []

# tp = []
# fp = []

# tn = []
# fn = []

# if os.path.isfile(val_result):
#     resultDF = pd.read_csv(val_result)
# else:
#     for plc in labPlace_tok.classes_:
#         SM_RE = SM_Reasons(plc)
#         EM_RE = EM_Reasons(plc, SM_RE)

#         # if plc == 'Sudan':
#         #     print(EM_RE)

#         # if len(EM_RE[0])+len(EM_RE[1]) == 0:
#         #     continue

#         # if len(SM_RE[0]) > 0:
#         temp_tp = len(EM_RE[0])
#         temp_fp = len(EM_RE[1])
#         # else:
#         #     continue

#         # if len(EM_RE[2])+len(EM_RE[3]) == 0:
#         #     continue

#         # if len(SM_RE[1]) > 0:
#         temp_tn = len(EM_RE[2])
#         temp_fn = len(EM_RE[3])
#         # else:
#         #     continue

#         country.append(plc)

#         true_positive.append(EM_RE[0])
#         false_positive.append(EM_RE[1])
#         true_negative.append(EM_RE[2])
#         false_negative.append(EM_RE[3])

#         tp.append(temp_tp)
#         fp.append(temp_fp)
#         tn.append(temp_tn)
#         fn.append(temp_fn)

#     resultDict = {'Country': country, 'True Positive': true_positive, 'False Positive': false_positive, 'True Negative': true_negative, 'False Negative': false_negative, 'TP': tp, 'FP': fp, 'TN': tn, 'FN': fn}
#     resultDF = pd.DataFrame(data=resultDict)
#     resultDF.to_csv(val_result)

In [92]:
# TP = resultDF['TP'].sum()
# FP = resultDF['FP'].sum()
# TN = resultDF['TN'].sum()
# FN = resultDF['FN'].sum()

In [93]:
# Accuracy = ((TP+TN) / (TP+FP+FN+TN))
# Precision = (TP / (TP+FP))
# Recall = (TP / (TP+FN))
# F1_Score = 2*(Recall * Precision) / (Recall + Precision)

In [94]:
# Accuracy = round(Accuracy * 100, 2)
# Precision = round(Precision * 100, 2)
# Recall = round(Recall * 100, 2)
# F1_Score = round(F1_Score * 100, 2)

In [95]:
# print(f'Accuracy \t: {Accuracy} %')
# print(f'Precision \t: {Precision} %')
# print(f'Recall \t\t: {Recall} %')
# print(f'F1 Score \t: {F1_Score} %')